In [1]:
import sys
from sklearn.externals import joblib
from user_function import MyAlgorithm
from alcon_utils import AlconUtils
import numpy as np
import cv2

import keras
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import tensorflow
import cProfile, pstats

import os.path
import pickle

config = tensorflow.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.93
keras.backend.tensorflow_backend.set_session(tensorflow.Session(config=config))

# 初期化
datasetdir = "/share/alcon/dataset/"
alcon = AlconUtils(datasetdir)

Using TensorFlow backend.


In [45]:
pr = cProfile.Profile()
pr.enable()

# アノテーションの読み込み
alcon.load_annotations_target("target_lv1_iiyama_0.9.csv")
alcon.load_annotations_ground("groundtruth_lv1_iiyama_0.9.csv")

dataset = {}
for bb_id, target in alcon.targets.items():
    code = alcon.ground_truth[bb_id][0]
    if code not in dataset:
        dataset[code] = []
    if len(dataset[code]) == 300:
        continue
    img_filename = alcon.get_filename_char( bb_id )
    img = cv2.imread( img_filename )
    feature = MyAlgorithm.feature_extraction(img)
    dataset[code].append(feature)

labels = []
data = []
classes = sorted(dataset.keys())

for label, values in dataset.items():
    labels += [classes.index(label)] * len(values)
    data += values

num_classes = 46
input_shape = (32, 32, 1)# img_rows img_cols channel

x_data = np.asarray(data).reshape(len(data), *input_shape)
y_train = keras.utils.to_categorical( labels, num_classes )
datasetdir = "/share/alcon/dataset/"
annotation_name = "test_5"

# 初期化
alcon = AlconUtils(datasetdir)
myalgorithm = MyAlgorithm(datasetdir)

# ターゲットの読み込み
file_name_last = "_lv1_" + annotation_name + ".csv"
alcon.load_annotations_target("target" + file_name_last)

pr.disable()
pstats.Stats(pr).sort_stats('tottime').print_stats(5)

         482548 function calls in 113.226 seconds

   Ordered by: internal time
   List reduced from 51 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    12998   89.744    0.007   89.744    0.007 {bilateralFilter}
    12998   22.272    0.002   22.272    0.002 {imread}
    12998    0.197    0.000    0.197    0.000 {cvtColor}
    12998    0.196    0.000    0.196    0.000 {threshold}
        1    0.171    0.171  113.007  113.007 <ipython-input-45-3403e1b54a1d>:9(<module>)




In [5]:
#　学習機に掛ける前の準備
pr = cProfile.Profile()
pr.enable()
config = tensorflow.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.93
keras.backend.tensorflow_backend.set_session(tensorflow.Session(config=config))

# 初期化
datasetdir = "/share/alcon/dataset/"
alcon = AlconUtils(datasetdir)

# アノテーションの読み込み
alcon.load_annotations_target("target_lv1_iiyama_0.9.csv")
alcon.load_annotations_ground("groundtruth_lv1_iiyama_0.9.csv")

dataset = {}
with open('full_dataset0.9', mode='rb') as f:
    dataset = pickle.load(f)
labels = []
data = []
classes = sorted(dataset.keys())

for label, values in dataset.items():
    labels += [classes.index(label)] * len(values)
    data += values

num_classes = 46
input_shape = (32, 32, 1)# img_rows img_cols channel
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    zca_whitening=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.3,
    zoom_range=0.1,
    fill_mode='constant',
    cval=0.)
x_train = np.asarray(data).reshape(len(data), *input_shape)
y_train = keras.utils.to_categorical( labels, num_classes )
datagen.fit(x_train)
datasetdir = "/share/alcon/dataset/"
annotation_name = "test_5"

# 初期化
alcon = AlconUtils(datasetdir)
myalgorithm = MyAlgorithm(datasetdir)

# ターゲットの読み込み
file_name_last = "_lv1_" + annotation_name + ".csv"
alcon.load_annotations_target("target" + file_name_last)

pr.disable()
pstats.Stats(pr).sort_stats('tottime').print_stats(5)

         863232 function calls (845262 primitive calls) in 2.916 seconds

   Ordered by: internal time
   List reduced from 1033 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.416    0.416    0.417    0.417 /root/.pyenv/versions/3.6.1/lib/python3.6/site-packages/scipy/linalg/decomp_svd.py:16(svd)
        3    0.210    0.070    0.210    0.070 {built-in method _pywrap_tensorflow_internal.TF_ExtendGraph}
      581    0.178    0.000    0.197    0.000 {built-in method numpy.core.multiarray.array}
        1    0.164    0.164    0.164    0.164 {built-in method _pickle.load}
        3    0.156    0.052    0.156    0.052 {built-in method numpy.core.multiarray.dot}




In [48]:
pr = cProfile.Profile()
pr.enable()
classifier = keras.models.Sequential()
classifier.add(keras.layers.normalization.BatchNormalization(input_shape = input_shape))
classifier.add(Conv2D(32, kernel_size=(3,3), activation='relu')) # 30*30
classifier.add(Conv2D(64,                 (3,3), activation='relu')) # 28*28 
classifier.add(Dropout(0.5))
classifier.add(MaxPooling2D(pool_size=(4,4)))                      # 7*7
classifier.add(Flatten())
classifier.add(Dense(480, activation='relu'))
classifier.add(Dropout(7/24))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(num_classes, activation='softmax'))

classifier.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Nadam(), metrics=['accuracy'])


classifier.fit(x_data, y_train, batch_size= 84, epochs=50,
                      callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)],
                      validation_data=(x_data, y_train))
classifier.fit_generator(datagen.flow(X_train, Y_train, batch_size=84),
                        samples_per_epoch=len(X_train),
                        nb_epoch=nb_epoch)

joblib.dump(classes, "./model.pkl")
classifier.save("./model2.pkl")





for bb_id, target in alcon.targets.items():
    img_file_id, *bb = target
    # 認識処理（ここで各自のアルゴリズムを呼び出します）
    # boundingbox id と紐付けて保存
    results[bb_id] = myalgorithm.predict(imgs[bb_id], bb)


# 評価
alcon.load_annotations_ground("groundtruth" + file_name_last)
alcon.evaluation( results )

pr.disable()
pstats.Stats(pr).sort_stats('tottime').print_stats(5)

TypeError: Error when checking model input: data should be a Numpy array, or list/dict of Numpy arrays. Found: <keras.preprocessing.image.ImageDataGenerator object at 0x7f5c1f9dc160>...

In [16]:
# テスト　imread
pr = cProfile.Profile()
pr.enable()
datasetdir = "/share/alcon/dataset/"
annotation_name = "test_5"
"""
評価コードのメイン
:param datasetdir データセットを格納したディレクトリへのパス
"""

# 初期化
alcon = AlconUtils(datasetdir)
myalgorithm = MyAlgorithm(datasetdir)

# ターゲットの読み込み
file_name_last = "_lv1_" + annotation_name + ".csv"
alcon.load_annotations_target("target" + file_name_last)

imgs = {}
results = {}
# １ターゲットずつ認識していくループ
for bb_id, target in alcon.targets.items():
    img_file_id, *bb = target
    # ページ全体の画像
    imgs[bb_id] = cv2.imread( os.path.join(datasetdir, "images", img_file_id+".jpg") )

pr.disable()
pstats.Stats(pr).sort_stats('tottime').print_stats(5)

         674686 function calls (656716 primitive calls) in 22.567 seconds

   Ordered by: internal time
   List reduced from 968 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      230   20.216    0.088   20.216    0.088 {imread}
        3    0.872    0.291    0.872    0.291 {built-in method _pywrap_tensorflow_internal.TF_ExtendGraph}
        5    0.266    0.053    0.266    0.053 {built-in method _pywrap_tensorflow_internal.TF_Run}
1105/1097    0.099    0.000    0.861    0.001 /root/.pyenv/versions/3.6.1/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:289(apply_op)
     1390    0.049    0.000    0.056    0.000 /root/.pyenv/versions/3.6.1/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:96(_extract_stack)




In [19]:
# テストpredict
pr = cProfile.Profile()
pr.enable()

datasetdir = "/share/alcon/dataset/"
"""
評価コードのメイン
:param datasetdir データセットを格納したディレクトリへのパス
"""

# 初期化
alcon = AlconUtils(datasetdir)
myalgorithm = MyAlgorithm(datasetdir)

# ターゲットの読み込み
alcon.load_annotations_target("target_lv1_test_5.csv")

for bb_id, target in alcon.targets.items():
    img_file_id, *bb = target
    # 認識処理（ここで各自のアルゴリズムを呼び出します）
    # boundingbox id と紐付けて保存
    results[bb_id] = myalgorithm.predict(imgs[bb_id], bb)


# 評価
alcon.load_annotations_ground("groundtruth" + file_name_last)
alcon.evaluation( results )

pr.disable()
pstats.Stats(pr).sort_stats('tottime').print_stats(5)

results: 195 / 230
score: 84.782609
         898856 function calls (880410 primitive calls) in 7.065 seconds

   Ordered by: internal time
   List reduced from 1002 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      230    3.099    0.013    3.099    0.013 {bilateralFilter}
        4    1.520    0.380    1.520    0.380 {built-in method _pywrap_tensorflow_internal.TF_ExtendGraph}
      235    0.841    0.004    0.841    0.004 {built-in method _pywrap_tensorflow_internal.TF_Run}
1106/1098    0.107    0.000    0.930    0.001 /root/.pyenv/versions/3.6.1/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:289(apply_op)
     1391    0.051    0.000    0.058    0.000 /root/.pyenv/versions/3.6.1/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:96(_extract_stack)




In [24]:
import pickle
with open('dataso', 'wb') as f:
    pickle.dump(dataset, f)

In [28]:
i = 0
for bb_id, target in alcon.targets.items():
    i += 1
    if i > 350*4: break
    img_file_id, *bb = target
    if i % 50 == 1: print(i)
    # ページ全体の画像
    imgs[bb_id] = cv2.imread( os.path.join(datasetdir, "images", img_file_id+".jpg") )

AttributeError: 'dict' object has no attribute 'shape'

In [46]:
x_data = datagen.reshape()

In [7]:
with open('full_dataset0.9', 'wb') as f:
    pickle.dump(dataset, f)

KeyboardInterrupt: 

KeyError: 4042